In [ ]:
import re
import os

# Fonction pour convertir les balises HTML en LaTeX
def convert_html_to_latex(text):
    text = re.sub(r'<p>(.*?)</p>', r'\\par \1 \\\\', text, flags=re.DOTALL)
    text = re.sub(r'<em>(.*?)</em>', r'\\emph{\1}', text)
    text = re.sub(r'<a href="(.*?)">(.*?)</a>', r'\2\\footnote{{\2 :\\url{\1}} }', text)
    text = text.replace('&', '\&')
    text = text.replace('[', '{[}')
    text = text.replace(']', '{]}')
    text = text.replace('<p>', '{}')
    text = text.replace('</p>', '{}')
    text = text.replace('§', '{}')
    text = text.replace('#', '{}')
    text = text.replace('1 -', '{}')
    text = text.replace('1-', '{}')
    text = text.replace('2 -', '{}')
    text = text.replace('3 -', '{}')
    text = text.replace('4 -', '{}')
    text = text.replace('5 -', '{}')
    text = text.replace('<span>', '{}')
    text = text.replace('</span>', '{}')
    text = text.replace('<br/>', '{}')
    
    return text

# Chemin vers le fichier HTML
html_file_path = 'records_data.html'

# Lire le contenu du fichier HTML
with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Commencer à construire le document LaTeX avec les nouvelles configurations
latex_contents = []

# Extraire les informations du HTML et les convertir en LaTeX
pattern = re.compile(r'<p>(.*?)<\/p>', re.DOTALL)
matches = pattern.findall(html_content)

for match in matches:
    text = match.strip()
    text = convert_html_to_latex(text)

    if 'Cite As:' in text:
        text = text.replace('Cite As:', '\\textbf{Cite As:}')
        url_match = re.search(r'(https?://\S+)', text)
        if url_match:
            url = url_match.group(0)
            text = text.replace(url, f'\\href{{{url}}}{{{url}}}')
    else:
        if 'Type:' in text:
            text = '\\newpage\n' + text

        if ':' in text:
            key, value = text.split(':', 1)
            text = f'\\textbf{{{key.strip()}:}} {value.strip()}'
        else:
            text = text.replace('AJOUTER UN TITRE', '\\textbf{AJOUTER UN TITRE}')

    latex_contents.append(text)

# Déterminer combien de parties nous avons besoin
num_parts = 1
total_texts = len(latex_contents)
texts_per_part = total_texts // num_parts

# Chemin de sortie pour les fichiers LaTeX
output_directory = 'D:/result_heursit/'

# Créer le répertoire de sortie s'il n'existe pas
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Écrire chaque partie dans un fichier LaTeX séparé
for i in range(num_parts):
    start_index = i * texts_per_part
    end_index = (i + 1) * texts_per_part if i < num_parts - 1 else total_texts
    latex_part = "\n".join(latex_contents[start_index:end_index])

    # Nom du fichier de sortie
    latex_output_path = f'{output_directory}records_data_part_{i+1}.tex'

    # Commencer le contenu LaTeX pour le premier fichier avec la page de titre et l'introduction
    if i == 0:
        latex_content = r"""
\documentclass{book}
\usepackage[top=2.5cm, bottom=2.5cm, left=2.5cm, right=2.5cm]{geometry}
\usepackage{graphicx}
\usepackage{multicol}
\usepackage[utf8]{inputenc}  % Encodage UTF-8
\usepackage[francais]{babel} % Langue française
\usepackage[center, large]{titlesec}
\usepackage{indentfirst}
\usepackage{soul}
\usepackage{pdfpages}
\usepackage{xunicode}
\usepackage{xltxtra}
\usepackage{scrextend}
\usepackage{fancyhdr}
\usepackage{hyperref}
\setmainfont{Times New Roman}
\usepackage{parskip}

% En-têtes et pieds de page
\pagestyle{fancy}
\fancyhf{}
\fancyhead[LE,RO]{}
\fancyhead[RE,LO]{\textit{Thèse de Clément}}
\fancyfoot[LE,RO]{\thepage}
\fancyfoot[RE,LO]{}

% Début du document
\begin{document}

% Page de titre
\title{Thèse Clément heurist}
\author{Clément Dussart}
\date{}
\maketitle

% Page d'introduction
\newpage
\section*{Introduction}
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vivamus lacinia odio vitae vestibulum vestibulum. Cras venenatis euismod malesuada.
"""
    else:
        latex_content = r"""
\documentclass{book}
\usepackage[top=2.5cm, bottom=2.5cm, left=2.5cm, right=2.5cm]{geometry}
\usepackage{graphicx}
\usepackage{multicol}
\usepackage[utf8]{inputenc}  % Encodage UTF-8
\usepackage[francais]{babel} % Langue française
\usepackage[center, large]{titlesec}
\usepackage{indentfirst}
\usepackage{soul}
\usepackage{pdfpages}
\usepackage{xunicode}
\usepackage{xltxtra}
\usepackage{scrextend}
\usepackage{fancyhdr}
\usepackage{hyperref}
\setmainfont{Times New Roman}

% En-têtes et pieds de page
\pagestyle{fancy}
\fancyhf{}
\fancyhead[LE,RO]{}
\fancyhead[RE,LO]{\textit{Thèse de Clément}}
\fancyfoot[LE,RO]{\thepage}
\fancyfoot[RE,LO]{}

% Début du document
\begin{document}
"""

    # Ajouter le contenu de cette partie avec les modifications pour \textbf
    for line in latex_part.splitlines():
        if line.startswith("\\textbf{"):
            latex_content += "\n" + line + "\n"
        else:
            latex_content += line + "\n"

    # Ajouter les séparateurs de section
    latex_content += "\\hrulefill\n"

    # Terminer le document LaTeX
    latex_content += "\\end{document}"

    # Écrire le contenu dans le fichier LaTeX
    with open(latex_output_path, 'w', encoding='utf-8') as file:
        file.write(latex_content)

    print(f"Document LaTeX créé avec succès : {latex_output_path}")
